In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Tuesday, July 30, 2024
Last Modified: Wednesday, August 20, 2025
Program Purpose: To create a plot which shows all of the individual carbon flux contribution values across the models as well as the median of
the models.
"""

In [ ]:
#Importing the appropriate libraries
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import glob
import netCDF4 as nc
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

#Making font times
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'

In [ ]:
# Area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')

#The ordered dict to hold the results
all_datasets_std = OrderedDict()

#Loading and processing all of the files
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "S3_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.upper()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value.
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented.
    dataset.close()

    # Removing the unduely high values from the Classic dataset
    if file_name == "CLASSIC-":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero.

    # Removing the water values from the yibs that are not blocked out
    if file_name == "YIBS-":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0)
        mask = data_modis_52 != 0
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)

    # Finding the STD of each grid cell over the years
    standard_deviation_array = np.std(terrestrial_flux, axis=0)
    
    # Adding the placeholder array to the list of IAV maps.
    all_datasets_std[file_name] = standard_deviation_array




In [ ]:
#This block is concerned with the generation of the SD maps for all of the models

#Masking the coasts with the MODIS data
#Masking out the coastal areas to be aligned with the MODIS data
modis_data = np.load("MODIS_Map.npy")
for model in all_datasets_std:
    all_datasets_std[model][modis_data == 0] = np.nan

# Create the figure and subplots
fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(11, 12.5), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten() #Flattening the axes to more easily cycle through

#Cycling the axes to add the maps to the plot altogether
i = 0
for model in all_datasets_std:
    image = axes[i].imshow(all_datasets_std[model], cmap='Reds', origin='upper', extent=[-180, 180, -90, 90])
    axes[i].set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded
    #Ading the map features
    axes[i].add_feature(cfeature.BORDERS, linestyle=':')
    axes[i].add_feature(cfeature.COASTLINE)
    # Add a title to each subplot
    temp_title = str(model).replace("_", "-").upper().replace("-S3", "")
    temp_title = temp_title.strip("-")
    axes[i].set_title(f"{temp_title}", fontsize=25)
    i+=1


# Adding the colorbar to the left
axes_cbar = fig.add_axes([0,0,0.01,1]) #Location of the colorbar
cbar = fig.colorbar(image, cax=axes_cbar) #Adding the colorbar
cbar.ax.ticklabel_format(style='sci', scilimits=(0,0), axis='y')
cbar.ax.yaxis.offsetText.set_fontsize(25)  
cbar.ax.yaxis.set_ticks_position('left')
cbar.ax.yaxis.set_label_position('left')
cbar.ax.tick_params(labelsize=25, width=3)
cbar.set_label(r'$\sigma$ (Annual Carbon Flux) ($\mathrm{PgC} \ \mathrm{yr^{-1}}$) (1970-2020)', fontsize=25)

#Tightening up the plot
plt.tight_layout()
plt.show()


In [ ]:
#This block is concerned with the creation of the median map of the SD across the models

#Stacking and then taking the median
list_data = []
for models in all_datasets_std:
    list_data.append(all_datasets_std[models])

#Stacking
stacked_data = np.stack(list_data)

#Compiling the median
median_data = np.ma.median(stacked_data, axis=0)

#Checking the shape of the map
print(np.shape(median_data))

#Printing out the map
fig, ax = plt.subplots(figsize=(40,20), subplot_kw={'projection': ccrs.PlateCarree()})
image = ax.imshow(median_data, cmap='Reds', origin='upper', extent=[-180, 180, -90, 90])
ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded
#Ading the map features
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
gl.left_labels = False
gl.top_labels = False
gl.ylabel_style = {'size': 45}
gl.xlabel_style = {'size': 45}
#Adding the colorbar
cbar = plt.colorbar(image, ax=ax, orientation='vertical', pad=0.02)
cbar.ax.set_position([ax.get_position().x0 - 0.02, ax.get_position().y0, 0.02, ax.get_position().height]) #Seting the alignment in line with the map
cbar.ax.ticklabel_format(style='sci', scilimits=(0,0), axis='y')
cbar.ax.yaxis.offsetText.set_fontsize(60)  
cbar.ax.yaxis.set_ticks_position('left')
cbar.ax.yaxis.set_label_position('left')
cbar.ax.tick_params(labelsize=60, width=3)
cbar.set_label(r'$\sigma$ (Annual Carbon Flux)' '\n' r'($\mathrm{PgC} \, \mathrm{yr^{-1}}$) (1970-2020)', fontsize=60)

